# Loteca

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Ler arquivo

In [2]:
columns = ["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]

In [3]:
df1 = pd.read_csv("./arquivos/Italy/I1.csv", parse_dates=["Date"])[columns]
for i in range(1,14):
    print(i)
    df1 = df1.append(pd.read_csv("./arquivos/Italy/I1 ("+str(i)+").csv", parse_dates=["Date"])[columns])

1
2
3
4
5
6
7
8
9
10
11
12
13


In [4]:
df1 =  df1.sort_values('Date', ascending = False).reset_index(drop=True).dropna()

In [ ]:
df1.to_csv("./arquivos/Italy/italy_geral.csv")

In [ ]:
df1 = pd.read_csv("./arquivos/Italy/italy_geral.csv", parse_dates=["Date"])

In [ ]:
# Div = League Division
# Date = Match Date (dd/mm/yy)
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG = Full Time Home Team Goals
# FTAG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards

In [5]:
# Compute points won for each match

# df1 =  df1.sort_values('Date', ascending = False).reset_index(drop=True)

for ind, row in df1.iterrows():
    df1.at[ind,'ANO_MES'] = str(row['Date'].year) + '/' + str(row['Date'].month)
    if row['Date'].month >= 8:
        df1.at[ind,'SEASON'] = str(row['Date'].year) + '/' + str(row['Date'].year + 1)
    
    if row['Date'].month <= 5:
        df1.at[ind,'SEASON'] = str(row['Date'].year - 1 ) + '/' + str(row['Date'].year)
    
    if row['FTR'] == 'H':
        df1.at[ind,'HPoints'] = 3
        df1.at[ind,'APoints'] = 0
    else:
        if row['FTR'] == 'A':
            df1.at[ind,'HPoints'] = 0
            df1.at[ind,'APoints'] = 3            
        else:
            if row['FTR'] == 'D':
                df1.at[ind,'HPoints'] = 1
                df1.at[ind,'APoints'] = 1

In [6]:
# Compute goals sum for the least 10 games

# df1 =  df1.sort_values('Date', ascending = False).reset_index(drop=True)

for ind in range(len(df1)):
    
    
    HG = df1[ind:][df1['HomeTeam'] == df1.iloc[ind]['HomeTeam']][1:11].sum()[['FTHG', 'FTAG', 'HST', 'HC']]
    
    df1.at[ind,'HScoredGSum'] = HG['FTHG']
    df1.at[ind,'HConseivedGSum'] = HG['FTAG']
    df1.at[ind,'SumHST'] = HG['HST']
    df1.at[ind,'SumHC'] = HG['HC']
    
    
    
    AG = df1[ind:][df1['AwayTeam'] == df1.iloc[ind]['AwayTeam']][1:11].sum()[['FTHG', 'FTAG', 'AST', 'AC']]
   
    df1.at[ind,'AScoredGSum'] = AG['FTAG']
    df1.at[ind,'AConseivedGSum'] = AG['FTHG']
    df1.at[ind,'SumAST'] = AG['AST']
    df1.at[ind,'SumAC'] = AG['AC']

    

    

C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [7]:
df1.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,HPoints,APoints,HScoredGSum,HConseivedGSum,SumHST,SumHC,AScoredGSum,AConseivedGSum,SumAST,SumAC
0,I1,2018-12-05,Benevento,Genoa,1.0,0.0,H,14.0,13.0,3.0,...,3.0,0.0,17.0,20.0,64.0,63.0,10.0,15.0,29.0,31.0
1,I1,2018-12-05,Inter,Sassuolo,1.0,2.0,A,16.0,13.0,8.0,...,0.0,3.0,18.0,8.0,69.0,71.0,9.0,14.0,46.0,42.0
2,I1,2018-11-26,Cagliari,Torino,0.0,0.0,D,3.0,8.0,1.0,...,1.0,1.0,10.0,8.0,36.0,56.0,11.0,13.0,35.0,44.0
3,I1,2018-11-25,Genoa,Sampdoria,1.0,1.0,D,10.0,6.0,5.0,...,1.0,1.0,13.0,14.0,45.0,48.0,13.0,19.0,35.0,51.0
4,I1,2018-11-25,Parma,Sassuolo,2.0,1.0,H,10.0,12.0,4.0,...,3.0,0.0,10.0,9.0,41.0,32.0,10.0,12.0,45.0,41.0


In [8]:
for ind in range(len(df1)):
    
    df1.at[ind,'RankH'] = df1[ind:][(df1['HomeTeam'] == df1.iloc[ind]['HomeTeam']) & (df1['SEASON'] == df1.iloc[ind]['SEASON'])][1:].sum()['HPoints'] + \
    df1[ind:][(df1['AwayTeam'] == df1.iloc[ind]['HomeTeam']) & (df1['SEASON'] == df1.iloc[ind]['SEASON'])][:].sum()['APoints']
    
    
    df1.at[ind,'RankA'] = df1[ind:][(df1['AwayTeam'] == df1.iloc[ind]['AwayTeam']) & (df1['SEASON'] == df1.iloc[ind]['SEASON'])][1:].sum()['APoints'] + \
    df1[ind:][(df1['HomeTeam'] == df1.iloc[ind]['AwayTeam']) & (df1['SEASON'] == df1.iloc[ind]['SEASON'])][:].sum()['HPoints']



C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [9]:
df1.to_csv("./arquivos/Italy/italy_geral_computed_indices.csv")

In [64]:
df1 = pd.read_csv("./arquivos/Italy/italy_geral_computed_indices.csv", parse_dates=["Date"])

In [ ]:
df1[df1['SEASON'] == '2017/2018'].sort_values('RankH',  ascending = False).head(10)

### Seleção de colunas

In [82]:
# columns_x = ["AVG_FTHG","AVG_HS","AVG_HST","AVG_HF","AVG_HC","AVG_HY","AVG_HR","AVG_FTAG","AVG_AS","AVG_AST","AVG_AF","AVG_AC","AVG_AY","AVG_AR"]
columns_x = ['HScoredGSum', 'SumHST', 'SumHC', 'RankH','AScoredGSum', 'SumAST', 'SumAC', 'RankA']

df1 =  df1.sort_values('Date', ascending = True).reset_index(drop=True).dropna()

df1 = df1.dropna()

# X = df_medias[columns_x]
X = df1[columns_x]
y = df1['FTR']

### Suport vector machine

In [69]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
FTR = len(df1)/(df1.groupby('FTR').count()['Div'])

# FTR = FTR/FTR.max()

weight = {frk: FTR[frk] for frk in FTR.keys()}

In [70]:
# model = MultiOutputClassifier(RandomForestClassifier())
model = LogisticRegression()


### Separar os datasets de treinamento e teste

In [83]:
split = 40
X_train, y_train, X_test, y_test = X[:-int(np.array(X).shape[0]/split)], y[:-int(np.array(y).shape[0]/split)], X[-int(np.array(X).shape[0]/split):], y[-int(np.array(y).shape[0]/split):]


In [72]:
# Standardising the data.
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize

X_train = normalize(X_train)
X_test = normalize(X_test)

In [73]:
model.fit(X_train, y_train)

C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Miniconda3\envs\loteca\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### Avaliar precisão

In [74]:
y_pred = model.predict(X_test)

In [87]:
for i in range(len(X_test)):
    if X_test[i,7] > X_test[i,3]:
        y_pred[i] = 'A'
    else:
        if X_test[i,7] == X_test[i,3]:
            y_pred[i] = 'D'
        else:
            y_pred[i] = 'H'

KeyError: (0, 7)

In [84]:
X_test

,HScoredGSum,SumHST,SumHC,RankH,AScoredGSum,SumAST,SumAC,RankA
4590,14.0,43.0,32.0,0.0,10.0,35.0,49.0,0.0
4591,10.0,40.0,48.0,0.0,9.0,30.0,40.0,0.0
4592,12.0,47.0,76.0,0.0,8.0,25.0,44.0,0.0
4593,25.0,58.0,47.0,3.0,22.0,55.0,56.0,3.0
4594,23.0,86.0,69.0,6.0,12.0,52.0,59.0,1.0
4595,11.0,32.0,47.0,3.0,10.0,31.0,39.0,1.0
4596,5.0,40.0,47.0,0.0,3.0,26.0,35.0,0.0
4597,14.0,42.0,68.0,0.0,10.0,43.0,54.0,3.0
4598,8.0,31.0,54.0,0.0,9.0,40.0,43.0,4.0
4599,13.0,50.0,68.0,0.0,8.0,27.0,40.0,0.0


In [85]:
r = []
for i in range(len(y_pred)):   
    if y_pred[i] == y_test.iloc[i]:        
        r.append(1)
    else:
        r.append(0)   
    

In [86]:
np.array(r).mean()

0.3162393162393162

In [77]:
y_pred

array(['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'A', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
      dtype=object)

### Salvar o modelo

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(model, 'model.italy')

In [ ]:
model = joblib.load('model.italy') 

### Lista de times

In [ ]:
vocab = np.unique(np.concatenate((np.array(df1['HomeTeam'].unique()), np.array(df1['AwayTeam'].unique()))))

In [ ]:
vocab

In [ ]:
# columns_x = ['HPoints','APoints','HScoredGSum','HConseivedGSum','AScoredGSum','AConseivedGSum']

def prepare(H,A):
    ret = []
    retH = df1[df1['HomeTeam'] == H].iloc[0][['HPoints','HScoredGSum','HConseivedGSum']]
    retA = df1[df1['AwayTeam'] == A].iloc[0][['APoints','AScoredGSum','AConseivedGSum']]
    return [retH['HPoints'], retA['APoints'], retH[['HScoredGSum','HConseivedGSum']], retA[['AScoredGSum','AConseivedGSum']]]   
    
    

In [ ]:
prepare('Ascoli', 'Atalanta')

### Realizar previsão

In [ ]:
H = 'Ceara'
A = 'Atletico-MG'

p = model.predict([prepare(H,A)])

if p < 0:
    print(H, 'Vence com ', round(abs(p[0]),3),' gol de diferença')
else:
    print(A, 'Vence com', round(abs(p[0]),3),' gol de diferença')